# 1. Load Data

In [ ]:
import random
random.seed(8)

# Import libraries
!pip install pandas matplotlib statsmodels pmdarima keras tensorflow yellowbrick

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pmdarima as pm
import math
import os
import warnings
warnings.filterwarnings('ignore')

from math import sqrt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import het_arch
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from pylab import rcParams

from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

import tensorflow as tf
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False # to hide warnings during training stage

#Import libraries for kmeans clustering model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics

# Define a customized palette for future plots
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60", "#D6B0CA"]
cmap = ListedColormap(pal)
palette = pal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import dataset
data = pd.read_csv('/content/drive/MyDrive/[H4TF DA] Round 2_MLTada/100000 Sales Records.csv')

# 2. EDA

The data set includes 14 data fields, detailed below:

| Feature               | Data Type | Description                                                      |
|:----------------------|:----------|:-----------------------------------------------------------------|
| Region                | object    | Geographic region of the sale                                    |
| Country               | object    | Country where the sale occurred                                  |
| Item Type             | object    | Category of the item sold                                        |
| Sales Channel         | object    | Channel through which the sale was made (Online/Offline)         |
| Order Priority        | object    | Priority level of the order (L/M/H/C)                            |
| Order Date            | datetime  | Date when the order was placed                                   |
| Order ID              | int       | Unique identifier for the order                                  |
| Ship Date             | datetime  | Date when the order was shipped                                  |
| Units Sold            | int       | Number of units sold                                             |
| Unit Price            | float     | Price per unit of the item                                       |
| Unit Cost             | float     | Cost per unit of the item                                        |
| Total Revenue         | float     | Total revenue from the sale                                      |
| Total Cost            | float     | Total cost incurred for the sale                                 |
| Total Profit          | float     | Total profit from the sale                                       |


In [ ]:
# Check the first 5 rows of the data set
data.head()

In [ ]:
# Check the data type of data fields
data.info()

However, at the time the data is imported into the ipynb file, the data fields still have the datatype object. Therefore, the team will clean the data before proceeding with further processing.

## 2.1. Cleaning Data

The data cleaning process includes converting the datatype of the columns to the appropriate types.

In [ ]:
# Change order date, ship date to datetime
data['Order Date']=pd.to_datetime(data['Order Date'])
data['Ship Date']=pd.to_datetime(data['Ship Date'])

In [ ]:
# Define the desired column order
new_order = ['Order ID', 'Item Type', 'Sales Channel', 'Order Priority', 'Country', 'Region', 'Order Date', 'Ship Date', 'Units Sold', 'Unit Price', 'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit']
# Reorder the DataFrame
data = data[new_order]
data.head()

In [ ]:
# Gross profit margins
data['Gross Profit Margin'] = data['Total Profit'] / data['Total Revenue']*100
# Return on investment
data['ROI']= data['Total Profit'] / data['Total Cost']*100

data.head()

In [ ]:
#Round column Gross Profit Margin and ROI
data['Gross Profit Margin']=data['Gross Profit Margin'].round(2)
data['ROI']=data['ROI'].round(2)
data.head()

In [ ]:
#Order by Order Date
data=data.sort_values(by='Order Date')
data.head()

In [ ]:
# Box plot to check outlier
data['Year'] = data['Order Date'].dt.year
sns.boxplot(x='Year', y='Total Revenue', data=data)
plt.title('Total Revenue by Year')
plt.xlabel('Year')
plt.ylabel('Total Revenue')
plt.show()
# Count outliers each year


As can be seen from the plot above, there's a considerable amount of outliers in terms of yearly revenue. For this reason, we decided to filter these outliers from our dataset, determining the existence of these outliers as 'Agents' which poses unnecessary impacts on our predictions.

In [ ]:
'''DISPLAY OUTLIER OF 'TOTAL REVENUE' '''
def display_outliers(data):
  grouped = data.groupby('Year')['Total Revenue'].describe()
  grouped['IQR'] = grouped['75%'] - grouped['25%']
  grouped['Lower Bound'] = grouped['25%'] - 1.5 * grouped['IQR']
  grouped['Upper Bound'] = grouped['75%'] + 1.5 * grouped['IQR']


  outliers_count = []
  for year in data['Year'].unique():
      year_data = data[data['Year'] == year]
      outliers = year_data[(year_data['Total Revenue'] < grouped.loc[year, 'Lower Bound']) |
                          (year_data['Total Revenue'] > grouped.loc[year, 'Upper Bound'])]
      outliers_count.append((year, len(outliers)))

  for year, count in outliers_count:
      print(f"Number of outliers in {year}: {count}")

'''REMOVE OUTLIERS'''
def remove_rev_outliers(data):

    grouped = data.groupby('Year')['Total Revenue'].describe()
    grouped['IQR'] = grouped['75%'] - grouped['25%']
    grouped['Lower Bound'] = grouped['25%'] - 1.5 * grouped['IQR']
    grouped['Upper Bound'] = grouped['75%'] + 1.5 * grouped['IQR']

    filtered_data = pd.DataFrame()
    for year in data['Year'].unique():
        year_data = data[data['Year'] == year]
        valid_data = year_data[year_data['Total Revenue'].between(grouped.loc[year, 'Lower Bound'], grouped.loc[year, 'Upper Bound'], inclusive="both")]
        filtered_data = pd.concat([filtered_data, valid_data])

    return filtered_data

'''RECURSIVE FUNCTION TO REMOVE ALL OUTLIERS'''
def remove_all_outliers(data):
    while True:
        new_data = remove_rev_outliers(data)
        if len(new_data) == len(data):  # No outliers removed in this iteration
            break
        data = new_data

    return data

df = remove_all_outliers(data)
display_outliers(df)

In [ ]:
def extract_month(month_year):
        return pd.to_datetime(month_year, format='%Y-%m').month

df['Month-Year'] = df['Order Date'].dt.strftime('%Y-%m')

In [ ]:
df

In [ ]:
# Drop ID, Order and Ship Date (irrelevant)
drops = ['Order ID', 'Order Date', 'Ship Date']
df = df.drop(drops, axis=1)

## 2.2. Exploring Dataset and Explanation

### 2.2.1. Check the quality of the dataset

The dataset does not include null values, does not possess duplicate values, and the date column is completely unique. From there, in terms of quality, the data set has no problem.

In [ ]:
# Check the number of rows and columns of the data set
df.shape

In [ ]:
# Check the number of null observations
print(df.isnull().sum())

In [ ]:
# Check the number of duplicate observations
print("Duplicated observations: " + df.duplicated().sum().astype(str))

In [ ]:
# Check the number of unique values ​​of each data field
df.nunique()

### 2.2.2. Check the fields in the dataset

####2.2.2.1. Descriptive Analysis


- Descriptive Statistics data fields in a data set


In [ ]:
df.describe()

The authors examine the dataset and gain insights into the "Units Sold" variable. On average, 4528 units are sold, with a significant variation indicated by a standard deviation of 2839 units.

To better understand the distribution of units sold, the authors look at the quartiles. The median number of units sold is 4222.5, showing that half of the sales are below this amount while the other half exceeds it. Notably, there are outliers, with units sold ranging from a minimum of 1 to a maximum of 10000. These extremes may result from specific company circumstances, highlighting the diverse issues that businesses face.

### 2.2.3. Explore data fields

#### 2.2.3.1. Explore stock price data fields

In [ ]:
dfl = pd.melt(df[['Month-Year', 'Units Sold']], ['Month-Year'])

plt.figure(figsize=(16, 6))
sns.lineplot(data = dfl,
             x = 'Month-Year',
             y = 'value',
             hue = 'variable')
plt.title('Units Sold Over Time')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.show()

Based on the chart above, we can see that revenue fluctuated a lot from 2010 to 2017, and there must be many factors affecting this fluctuation. Let's analyze this fluctuation more closely and build a model to evaluate and predict the total units sold of the company.



####2.2.3.2: Univariate Analysis

In this section, our group will show the distribution chart of the four continuous variables : Total Revenue, Total Cost, Total Profit, Gross Profit Margin.

In [ ]:
features = ['Total Revenue', 'Total Cost', 'Total Profit', 'Gross Profit Margin']

fig, axes = plt.subplots(1, 4, figsize=(16, 6))
for i, feature in enumerate(features):
    sns.histplot(df[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()

The histograms reveal diverse financial landscapes within the dataset. Total revenue, total cost, and total profit show right-skewed distributions, indicating a concentration of lower values with a few outliers exhibiting exceptionally high figures. This suggests variability in company size and performance. Gross profit margin, however, displays a bimodal distribution with two distinct peaks, hinting at the existence of two distinct groups with differing profitability characteristics.|

####2.2.3.3. Bivariate Analysis

In [ ]:
# Select only numerical columns
numerical_columns = df.select_dtypes(include=[np.number])

# Drop 'Month-Year' column if it's still present (assuming it's datetime or string type)
if 'Month-Year' in numerical_columns.columns:
    numerical_columns = numerical_columns.drop(columns=['Month-Year'])

# Generate the correlation matrix
correlation_matrix = numerical_columns.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.show()


The relationship between the variables on the heatmap with positive correlation to Units Sold can be understood as follows:

- Units Sold and Total Revenue: The correlation coefficient is 0.43, indicating a weak relationship between units sold and total revenue. This suggests that while there is some association, the number of units sold is not the main factor driving total revenue.

- Units Sold and Total Cost: The correlation coefficient is 0.32, indicating a weak positive relationship. This suggests that an increase in units sold is slightly associated with an increase in total cost, but the relationship is not strong.

- Units Sold and Total Profit: The correlation coefficient is 0.56, indicating a moderate positive relationship. This suggests that as units sold increase, total profit also tends to increase, but other factors may also play a significant role.

- Units Sold and Gross Profit Margin: The correlation coefficient is 0.16, indicating a very weak positive relationship. This suggests that units sold have little to no impact on the gross profit margin.

- Units Sold and ROI: The correlation coefficient is 0.11, indicating a very weak positive relationship. This suggests that units sold have little to no impact on the return on investment (ROI).

Overall, the low correlation coefficients indicate that there is no strong relationship between these variables and units sold. In business analysis, this can be understood to mean that factors such as total revenue, total cost, total profit, gross profit margin, and ROI do not greatly influence the number of units sold. This may also reflect the complex nature of business dynamics, where other factors aside from these metrics play significant roles in influencing sales performance.

# 3. Model Building and Evalutions

## 3.1. K-Means Clustering

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/[H4TF DA] Round 2_MLTada/data for kmean.csv')
data.head()

,order_id,shipping_time,units_sold,total_revenue
0,753534924,26,6584,28785248
1,431350123,8,4942,126159376
2,737043845,6,6507,71108496
3,944893586,12,3865,258286355
4,959338033,7,8297,350042133


In [ ]:
sns.boxplot(x=data['units_sold'])
plt.show()

After building the boxplot of Units_Sold we see that the Units_Sold column has no Outliers.

Scale data (also known as feature scaling) is the process of normalizing data to bring them to the same value range. Typically, scaling data helps improve the performance of machine learning and data analysis algorithms. This is a very important step in building Model K-means Clustering

In [ ]:
data_kmean = data.copy()
cols_del = ['order_id']
data_kmean = data_kmean.drop(cols_del, axis=1)
scaler = StandardScaler()
scaler.fit(data_kmean)
scaled_data = pd.DataFrame(scaler.transform(data_kmean),columns= data_kmean.columns )
print("All features are now scaled")

In [ ]:
scaled_data.head()

The Elbow method is one of the popular methods to determine the optimal value of k in the K-Means clustering algorithm. This is a heuristics method used to determine the number of clusters in a data set.

In [ ]:
kmeans = KMeans(random_state=0)
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(), k=10)
Elbow_M.fit(scaled_data)
Elbow_M.show()

Using the Elbow Method, we were able to determine the most optimal number of clusters would be '5'.

In [ ]:
#Set random state for Model
random_state = 8
# Building K-means Clustering Model
k = 5

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(scaled_data)

data_kmean['Cluster'] = kmeans.labels_

print(data_kmean.head(15))
print(data_kmean['Cluster'].unique())

After our team chooses the appropriate Cluster parameters for the Model, the team proceeds to build the Model with the metrics Shipping Time, Units Sold and Total Revenue and devide Customer into 5 groups.

In [ ]:
data['clusters'] = kmeans.labels_

In [ ]:
data.head()

In [ ]:
#Distribution cluster
sns.countplot(x='clusters', data=data)
plt.show()

After classifying customers, we can see that group number 2 is a very small group and the largest group is group number 4. Let's analyze each group more closely in the next section.

In [ ]:
data.groupby(by='clusters').mean()

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for i in range(0, 5):
    sns.histplot(data[data['clusters'] == i]['units_sold'], ax=axes[i])
    axes[i].set_title('Cluster ' + str(i))
    axes[i].set_xlabel('Units Sold')

plt.tight_layout()
plt.show()

In [ ]:

fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for i in range(0, 5):
    sns.histplot(data[data['clusters'] == i]['shipping_time'], ax=axes[i])
    axes[i].set_title('Cluster ' + str(i))
    axes[i].set_xlabel('Shipping Time')
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:

fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for i in range(0, 5):
    sns.histplot(data[data['clusters'] == i]['total_revenue'], ax=axes[i])
    axes[i].set_title('Cluster ' + str(i))

plt.tight_layout()
plt.show()

The team chose the following names for the five customer groups after examining their attributes:
- Economical Occasional Buyers: This group purchases relatively little, generates little revenue, and experiences extremely slow shipping.
- High-Spending Patient Buyers: This customer segment has a medium spending level but a slow shipping time.
- High-Spending Impulse Buyers: Members of this group tend to make large, impulsive purchases, but their shipping times are erratic.
- Economical Fast Buyers: These are clients who spend little money and make modest purchases, but they have quick shipping.
- Medium-Spending Efficient Buyers: These are clients who make very large purchases, have a medium level of spending, and have very long shipping times.

## 3.2. LSTM Model

## Model Preprocessing


The model is preprocessed in these particular steps:
* Create a copy of the original data
* Sort the data by `Month-Year` for easy appending and splitting operations later on
* Select only the `Month-Year` and `Units Sold` columns as these are the columns we are interested in predicting
* Converting the DataFrame into a numpy array to fit within the LSTM model

As we are interested in building a model that can predict stock prices even in the circumstance that extreme external conditions are affecting the dataset, we decide to keep the whole set of data for prediction. Since although regular fluctuations can be seen throughout the period, the level of fluctuation and periodity remains the same. Future implementations might make a model with recent data seems more beneficial, but strict record in anomalies and data disruption might be required to determine that.

The data will then be split into `x_train` and `y_train` with the idea that for any values $i$ in `y_train`, the model will use values from $i - 1$ to $i - size$ to predict that particular value, or a sliding window approach.

The data is then converted into numpy arrays and reshaped to fit into the LSTM model.

## Model Training


We have tuned the data based on multiple rounds of trial and errors and we have decided on these hyperparameters:
* 2 layers model: The data is complex instead of exihibiting a stable trend so we will need 2 layers instead of 1.the
* `batch_size = 32`: This is generally considered  default value for `batch_size`.
* `optimizer = 'rmsprop'`: `rmsprop` is a well-known and widely used optimizer.
* `loss = 'mean_squared_error'`: This is used as MRSE is one of the criteria required by Business Case to evaluate the two models

## Model Testing

The test dataset is extracted and processed in a similar fashion as the traning


## Model Evaluation

Inheriting the requirements from the Business Case 1 prompt, this model will be evaluated based on 3 metricsL: R-squared, MAPE, and RMSE. The team has calculated these values for this model.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt


df_grouped = df.groupby('Month-Year')['Units Sold'].sum().reset_index() #group by sum of units sold per month-year
df_grouped['Month-Year'] = pd.to_datetime(df_grouped['Month-Year']) #convert month-year to datetime
df_grouped = df_grouped.sort_values(by=['Month-Year'])
#Set random state for Model
random_state = 8
# Filter out only 'Units Sold' column for the LSTM model and convert the DataFrame to a numpy array
HPG_subset = df_grouped[['Units Sold']]
HPG_LSTM = HPG_subset.values

# Train-Test Split
total_len = len(HPG_LSTM)
train_ratio = 0.8  # 80 Train - 20 Test
training_data_len = int(total_len * train_ratio)

# MinMaxScaler()
MMS = MinMaxScaler()
scaled_data = MMS.fit_transform(HPG_LSTM)

# Create the scaled training dataset
train_data = scaled_data[0:training_data_len, :]

x_train = []
y_train = []
size = 15

for i in range(size, len(train_data)):
    x_train.append(train_data[i-size:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

      # <LSTM model>
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dense(25))
model.add(Dense(1))

# Model compile and fit
model.compile(optimizer='rmsprop', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=32, epochs=30)

# Create the test dataset
test_data = scaled_data[training_data_len - size:, :]

# Split the data sets x_test and y_test
x_test = []
y_test = HPG_LSTM[training_data_len:, :]
for i in range(size, len(test_data)):
    x_test.append(test_data[i-size:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Predictions
predictions = model.predict(x_test)
predictions = MMS.inverse_transform(predictions)

# Metrics
r_squared = r2_score(y_test, predictions)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, predictions)
rmse = np.sqrt(np.mean((predictions - y_test) ** 2))

# Print metrics
LSTM_metrics = pd.DataFrame({
    'R-squared': [r_squared],
    'MAPE': [mape],
    'RMSE': [rmse]
}, index=['LSTM'])

print(LSTM_metrics)

# Plotting
viz_train = df_grouped[:training_data_len].copy()
viz_train.set_index('Month-Year', inplace=True)

viz_test = df_grouped[training_data_len:].copy()
viz_test.set_index('Month-Year', inplace=True)
viz_test.insert(1, 'Predictions', predictions, True)

# Predicting data in 1 year
n_steps = 12  # 12Months
x_input = train_data[-size:].reshape((1, size, 1))
future_predictions = []

for i in range(n_steps):
    yhat = model.predict(x_input, verbose=0)
    future_predictions.append(yhat[0, 0])
    x_input = np.roll(x_input, -1)
    x_input[0, (size-1), 0] = yhat

future_predictions = MMS.inverse_transform(np.array(future_predictions).reshape(-1, 1))
future_dates = pd.date_range(start=viz_test.index[-1], periods=n_steps+1, freq='M')[1:]
future_df = pd.DataFrame(future_predictions, index=future_dates, columns=['Future Predictions'])

plt.figure(figsize=(16, 6))
plt.plot(viz_train.index, viz_train['Units Sold'], label='Train')
plt.plot(viz_test.index, viz_test['Units Sold'], label='Test')
plt.plot(future_df.index, future_df['Future Predictions'], label='Future Predictions')
plt.title('Units Sold Prediction with LSTM Model')
plt.xlabel('Time')
plt.ylabel('Units Sold')
plt.legend(loc='upper left')
plt.show()

**Metrics Analysis**
- R-Squared (-0.087): On a scale of 1, a negative value would determine that the model is not doing a great job in predicting accurately.
- MAPE (2.8): A MAPE of 2.80% indicates that, on average, the model's predictions are off by 2.80%. This is a relatively low error rate and could be considered good
- RMSE (147130.95): A substantially high number for the Root Mean Square Error of the prediction comparing to actual data, indicating an inaccurate prediction.

**Model Optimization**
The provided metrics are, although indicative of the model’s malfunctioning, this shed light for further optimization techniques for this model.
 - Feature Engineering: Creating lagged variables to capture temporal dependencies, generating moving averages to identify trends, adding categorical features to account for cyclical patterns, and scaling or normalizing features to ensure they are on the same scale.
 - Different Algorithms: Algorithms such as Random Forests and Gradient Boosting can capture non-linear relationships and interactions between features. Support Vector Machines (SVMs) are effective for high-dimensional spaces and non-linear classification, while neural networks, including architectures like GRU or convolutional neural networks (CNNs), can be powerful for time series data.
 - Hyperparameter Tuning: Techniques such as grid search and random search systematically explore the hyperparameter space to find the best combination. Bayesian optimization offers a more efficient approach by using probabilistic models to balance exploration and exploitation. Cross-validation ensures that the tuned hyperparameters generalize well to unseen data.